In [ ]:
from PIL import Image, ImageOps
import io


def pil_from_base64(b64str):
    import base64
    header, b64 = b64str.split(',', 1) if ',' in b64str else (None, b64str)
    data = base64.b64decode(b64)
    img = Image.open(io.BytesIO(data)).convert('RGBA')
    bg = Image.new('RGBA', img.size, (255,255,255,255))
    merged = Image.alpha_composite(bg, img).convert('RGB')
    return merged


def preprocess_for_model(pil_img, size=(64,64)):
    # center-crop/pad to square and resize, return L mode (grayscale)
    return ImageOps.fit(pil_img, size).convert('L')

In [ ]:
# A tiny dummy model that chooses a mood randomly but with simple heuristic for visible pixels.
from PIL import ImageStat
import random


MOODS = ['happy','calm','sad','energetic']


class DummyMoodModel:
    def __init__(self):
        pass

    def predict_from_pil(self, pil_img):
        """Accepts a PIL grayscale image (64x64). Returns (mood, confidence)"""
        # heuristic: mean brightness
        stat = ImageStat.Stat(pil_img)
        mean = stat.mean[0]
        # mean near white -> calm/happy, near black -> energetic/sad randomly
        if mean > 220:
            mood = random.choice(['calm','happy'])
            conf = 0.6 + (mean - 220)/35 * 0.4
        elif mean > 120:
            mood = random.choice(['happy','energetic'])
            conf = 0.5 + (mean - 120)/100 * 0.5
        else:
            mood = random.choice(['sad','energetic'])
            conf = 0.55
        return mood, min(max(conf, 0.0), 0.99)